In [2]:
import os

In [4]:
%pwd

'/home/mohammadrasel/project/another/Kidney-Disease-Classification-Deep-Learning-Project/research'

In [5]:
os.chdir("../")

In [18]:
%pwd

'/home/mohammadrasel/project/another/Kidney-Disease-Classification-Deep-Learning-Project'

In [30]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:   
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    

In [31]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [32]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        
        prepare_base_model = self.config.prepare_base_model
        params=self.params
        
        training_model_data=os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([Path(training.root_dir)])
        
        training_config= TrainingConfig(

            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_model_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )
        return training_config
        
        

In [33]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [36]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    def get_base_model(self):
        self.model=tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    def train_valid_generator(self):
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.20
        )
        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:2],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        if self.config.params_is_augmentation:
            train_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                vertical_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                zoom_range=0.2,
                shear_range=0.2,
                **datagenerator_kwargs
            )
        else :
            train_datagenerator=valid_datagenerator
            
        self.train_generator=train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        
    def train(self):
        self.steps_per_epochs=self.train_generator.samples//self.train_generator.batch_size
        self.validation_steps=self.valid_generator.samples//self.valid_generator.batch_size
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epochs,
            validation_data=self.valid_generator,
            validation_steps=self.validation_steps
        )
        self.save_model(path=self.config.trained_model_path, model=self.model)
        

In [39]:
try:
    config=ConfigurationManager()
    training_config=config.get_training_config()
    print(training_config)
    training=Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e 

[2026-02-10 16:29:26,861: INFO: common: yaml file: config/config.yaml loaded successfully]
[2026-02-10 16:29:26,865: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-10 16:29:26,866: INFO: common: created directory at: artifacts]
[2026-02-10 16:29:26,868: INFO: common: created directory at: artifacts/training]
TrainingConfig(root_dir=PosixPath('artifacts/training'), trained_model_path=PosixPath('artifacts/training/trained_model.h5'), updated_base_model_path=PosixPath('artifacts/prepare_base_model/base_model_updated.h5'), training_data=PosixPath('artifacts/data_ingestion/kidney-ct-scan-image'), params_epochs=20, params_batch_size=16, params_is_augmentation=True, params_image_size=BoxList([224, 224, 3]))
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
Epoch 1/20


2026-02-10 16:29:27.615864: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


23/23 [==============================] - ETA: 0s - loss: 13.9175 - accuracy: 0.5730

2026-02-10 16:30:36.316771: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


23/23 [==============================] - 85s 4s/step - loss: 13.9175 - accuracy: 0.5730 - val_loss: 8.4312 - val_accuracy: 0.6000
Epoch 2/20
23/23 [==============================] - 84s 4s/step - loss: 8.0849 - accuracy: 0.6657 - val_loss: 6.5032 - val_accuracy: 0.4375
Epoch 3/20
23/23 [==============================] - 78s 3s/step - loss: 4.5197 - accuracy: 0.7275 - val_loss: 1.0342 - val_accuracy: 0.9250
Epoch 4/20
23/23 [==============================] - 78s 3s/step - loss: 3.4942 - accuracy: 0.7781 - val_loss: 0.7091 - val_accuracy: 0.6625
Epoch 5/20
23/23 [==============================] - 79s 3s/step - loss: 3.0347 - accuracy: 0.7837 - val_loss: 0.9627 - val_accuracy: 0.8375
Epoch 6/20
23/23 [==============================] - 81s 4s/step - loss: 2.8252 - accuracy: 0.7908 - val_loss: 3.2825 - val_accuracy: 0.4875
Epoch 7/20
23/23 [==============================] - 79s 3s/step - loss: 6.0686 - accuracy: 0.6713 - val_loss: 1.9620 - val_accuracy: 0.5125
Epoch 8/20
23/23 [============